<a href="https://colab.research.google.com/github/nyp-sit/aiup/blob/main/day2-pm/chatbot_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a ChatBot with Large Language Model

In this exercise, you will learn how to build a LLM-based chatbot.
We will explore two options:
1. using cloud-based services such as (Azure) Open AI servies or
2. using locally hosted open source model such as Llama

In [ ]:
%%capture
!pip install --upgrade gradio

## Option 1 - Using cloud API

We will be using Azure Open AI services in this lab exercise

In [ ]:
%%capture
!pip install openai

In [ ]:
from openai import AzureOpenAI

AZURE_ENDPOINT = "https://nypopenai2.openai.azure.com/"
API_KEY = "c0f0043bfe9b443eb7d02fc5edc525d2"

client = AzureOpenAI(
    api_key=API_KEY,
    api_version="2024-07-01-preview",
    azure_endpoint=AZURE_ENDPOINT)

In [ ]:
def get_completion_from_messages(messages, model="gpt-4o-global", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

In the code below, we first define context where the LLM will use to generate response. Context provides a reference (grounding) to guide LLM response.

In [ ]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages



def get_response(message, history):
    context.append({'role':'user', 'content':f"{message}"})
    response = client.chat.completions.create(
        model="gpt-4o-global",
        messages=context,
        temperature=0.0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
import gradio as gr

gr.ChatInterface(get_response, type="messages").launch(debug=True)

## Option 2 - Locally hosted LLM

We can use tools such as Ollama to run a supported LLM model. Ollama is a very popular serving platform to run large language models locally on your PC, and has optimized to accelerate the model's inference speed.

### Setting Up the Environment

 and use Ollama to run the Lllama model.
Ollama is a very popular serving platform to run large language models locally on your PC, and has optimized to accelerate the model's inference speed.
First, we need to set up our Colab notebook to support command-line operations, so that we can use command line to install the Ollama.



In [ ]:
!pip install colab-xterm


### Run Ollama using terminal

Run the following command in the terminal window to install Ollama:

```
curl https://ollama.ai/install.sh | sh
```

Start the Ollama server using the following command:

```
ollama serve &
```

The `&` at the end runs the command in the background, allowing you to continue using your terminal.

In [ ]:
%load_ext colabxterm
%xterm

### Pulling AI Models

Now that the Ollama server is running, we can pull AI models to use with our server. Let’s pull a Llama 3.2 1B parameters model as an example:

In [ ]:
%%capture
!ollama pull llama3.1:8b

### Interacting with Ollama using Python API

We will need to install Ollama python package to allow us to write code to interact with Ollama-hosted models.

In [ ]:
%%capture
!pip install ollama
!pip install jupyter_bokeh

In [ ]:
import ollama

response = ollama.chat(model='llama3.1:8b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  }], options = {'temperature': 0.0})
print(response['message']['content'])

In [ ]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
When you collects the order, \
asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, conversational and professional style. \
Once you have all the order details, just say bye and thank you. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


In [ ]:
response = ollama.chat(model='llama3.1:8b', messages=context)

print(response)

In [ ]:
import gradio as gr
import ollama

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

def get_response(message, history):

    context.append({'role':'user', 'content':f"{message}"})
    response = ollama.chat(model='llama3.1:8b', messages=context, options = {'temperature': 0.0})
    response_msg = response['message']['content']
    return response_msg


gr.ChatInterface(get_response, type="messages").launch(debug=True)